# About this kernel

The `cost_function` in this kernel is roughly 300x faster compared to the original kernel. Each function call takes roughly 37 µs.

## Reference

* (Excellent) Original Kernel: https://www.kaggle.com/inversion/santa-s-2019-starter-notebook
* First kernel that had the idea to use Numba: https://www.kaggle.com/nickel/250x-faster-cost-function-with-numba-jit
* Another great cost function optimization: https://www.kaggle.com/sekrier/fast-scoring-using-c-52-usec



In [1]:
import os

from numba import njit
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import random

## Read in the family information and sample submission

In [2]:
fpath = './Data/family_data.csv'
data = pd.read_csv(fpath, index_col='family_id')

fpath = './submission_71965.csv'
submission = pd.read_csv(fpath, index_col='family_id')

### Constants

In [3]:
N_DAYS = 100
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125

In [4]:
family_size = data.n_people.values
days_array = np.arange(N_DAYS, 0, -1)
choice_dict = data.loc[:, 'choice_0': 'choice_9'].T.to_dict()

In [5]:
choice_array_num = np.full((data.shape[0], N_DAYS + 1), -1)

for i, choice in enumerate(data.loc[:, 'choice_0': 'choice_9'].values):
    for d, day in enumerate(choice):
        choice_array_num[i, day] = d

In [6]:
penalties_array = np.array([
    [
        0,
        50,
        50 + 9 * n,
        100 + 9 * n,
        200 + 9 * n,
        200 + 18 * n,
        300 + 18 * n,
        300 + 36 * n,
        400 + 36 * n,
        500 + 36 * n + 199 * n,
        500 + 36 * n + 398 * n
    ]
    for n in range(family_size.max() + 1)
])

## Cost Function

In [7]:
@njit
def cost_function(prediction, penalties_array, family_size, days):
    penalty = 0

    # We'll use this to count the number of people scheduled each day
    daily_occupancy = np.zeros((len(days)+1))
    N = family_size.shape[0]
    
    # Looping over each family; d is the day, n is size of that family, 
    # and choice is their top choices
    for i in range(N):
        # add the family member count to the daily occupancy
        n = family_size[i]
        d = prediction[i]
        choice = choice_array_num[i]
        
        daily_occupancy[d] += n

        # Calculate the penalty for not getting top preference
        penalty += penalties_array[n, choice[d]]

    # for each date, check total occupancy
    #  (using soft constraints instead of hard constraints)
    relevant_occupancy = daily_occupancy[1:]
    incorrect_occupancy = np.any(
        (relevant_occupancy > MAX_OCCUPANCY) | 
        (relevant_occupancy < MIN_OCCUPANCY)
    )
    
    if incorrect_occupancy:
        penalty += 100000000

    # Calculate the accounting cost
    # The first day (day 100) is treated special
    init_occupancy = daily_occupancy[days[0]]
    accounting_cost = (init_occupancy - 125.0) / 400.0 * init_occupancy**(0.5)
    # using the max function because the soft constraints might allow occupancy to dip below 125
    accounting_cost = max(0, accounting_cost)
    
    # Loop over the rest of the days, keeping track of previous count
    yesterday_count = init_occupancy
    for day in days[1:]:
        today_count = daily_occupancy[day]
        diff = np.abs(today_count - yesterday_count)
        accounting_cost += max(0, (today_count - 125.0) / 400.0 * today_count**(0.5 + diff / 50.0))
        yesterday_count = today_count

    penalty += accounting_cost

    return penalty

## Simple Opimization Approach

For each family, loop over their choices, and if keep it if the score improves. There's a lot of easy improvement that can be made to this code.

In [8]:
# Start with the sample submission values
best = submission['assigned_day'].values
start_score = cost_function(best, penalties_array, family_size, days_array)

Let's see how fast it is:

In [ ]:
KT = 10
LastScore = 0
BestScore = start_score
for i in tqdm(range(0,500000)):
    new = best.copy()
    # loop over each family
    for fam_id in range(len(best)):
        # loop over each family choice
        for pick in range(10):
            day = choice_dict[fam_id][f'choice_{pick}']
            temp = new.copy()
            temp[fam_id] = day # add in the new pick
            prob = np.exp(-(cost_function(temp, penalties_array, family_size, days_array) - start_score)/KT)
            if np.random.rand() < prob:
                new = temp.copy()
                LastScore = start_score
                start_score = cost_function(new, penalties_array, family_size, days_array)
                
                    
                if abs(LastScore - start_score) < 0.0001:
                    KT = KT + 0.001
                else:
                    if(KT>1): KT=KT-0.5

    score = cost_function(new, penalties_array, family_size, days_array)
    submission['assigned_day'] = new
    if score < BestScore:
        print("New best : {}".format(int(score)))
        BestScore = score 
        submission.to_csv(f'./submissions/submission_{int(score)}.csv')

New best : 71958
New best : 71955
New best : 71955
New best : 71952
New best : 71949
New best : 71949
New best : 71947
New best : 71945
New best : 71945
New best : 71943
New best : 71941
New best : 71933
New best : 71931
New best : 71931
New best : 71916
New best : 71912
New best : 71909
New best : 71907
New best : 71903
New best : 71903
New best : 71902
New best : 71896
New best : 71894
New best : 71888
New best : 71884
New best : 71883
New best : 71851
New best : 71825
New best : 71824
New best : 71818
New best : 71817
New best : 71814
New best : 71810
New best : 71809
New best : 71802
New best : 71787
New best : 71784
New best : 71782
New best : 71777
New best : 71776
